In [2]:
import pandas as pd
df = pd.read_csv('RS.csv', low_memory=False)
#Formateo de las columnas
df.columns = [col.replace(' ', '_').lower() for col in df.columns]

#Cambio de formato y eliminacion de valores faltantes
Se reformateo la columna data_recorded y elimino dos filas que estaba inexistente 

In [3]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df.dropna(subset=['date_recorded'], axis = 0, inplace = True)

Antes de hacer un analisis, se elimino las propiedades que no iban acorde al objetivo 

In [4]:
df['property_type'].value_counts()

property_type
Single Family     401612
Residential       151064
Condo             105420
Two Family         26408
Three Family       12586
Vacant Land         7824
Commercial          5987
Four Family         2150
Apartments          1327
Industrial           795
Public Utility        10
Name: count, dtype: int64

In [5]:
#mascara de busqueda
sought_values = [ 'Public Utility', 'Commercial', 'Vacant Land' , 'Industrial', 'Apartments' ]
mask = df['property_type'].isin(sought_values)

# Eliminar las filas que cumplen la condición 
filtered_df = df.drop(df[mask].index, inplace = True)

Estas propiedades no son validas para un analisis de propiedades recidenciales, ya que no se clasifican como tal     

Se eliminaron los valores de Non use Code, ya podrian sesgar los datos, muchos de esos codigo refrejan transaciones no reales, las cuales podrian
llevanos a conclusiones erroneas 

In [6]:
#Eliminacion de filas y columnas 
df = df.drop(df[~df["non_use_code"].isnull()].index) 
df = df.drop("non_use_code", axis = 1)

Evaluamos la cantidad de propiedades y podemos notar que existe un dato llamado 'Residential'

In [7]:
df['property_type'].value_counts()

property_type
Single Family    279934
Residential      120534
Condo             77097
Two Family        13493
Three Family       5818
Four Family        1085
Name: count, dtype: int64

De manera visual podemos ver que en el residential_type nos especifica que tipo de propiedad es 

In [8]:
df[df['property_type'] == 'Residential'].sample(5)

,serial_number,list_year,date_recorded,town,address,assessed_value,sale_amount,sales_ratio,property_type,residential_type,assessor_remarks,opm_remarks,location
94800,210429,2021,2022-06-27,Newington,148 CHURCHILL DR,100270.0,205000.0,0.48910,Residential,Condo,NaN,NaN,POINT (-72.718352001 41.677475)
106791,210906,2021,2022-06-15,Southington,632 OVERLOOK PATH,179100.0,355000.0,0.50450,Residential,Condo,NaN,NaN,POINT (-72.862565998 41.599884979)
1094904,220095,2022,2023-02-21,Rocky Hill,207 HOLLY HILL DRIVE,144340.0,246000.0,0.58670,Residential,Condo,NaN,NaN,POINT (-72.702720004 41.663075016)
80171,210172,2021,2021-11-16,Hamden,2480 WHITNEY AVE U27,105560.0,195000.0,0.54130,Residential,Condo,NaN,NaN,POINT (-72.902021964 41.387668979)
89471,210891,2021,2022-02-03,Bridgeport,221 GODDARD AVE #223,194910.0,410000.0,0.47539,Residential,Three Family,NaN,NaN,POINT (-73.176883009 41.200257)


Podemos ver que valores estan relacionados, tranformando los valores de Residential a sus correspondiente en tipo de residencia 

In [9]:
df.groupby(['property_type', 'residential_type']).size()

property_type  residential_type
Condo          Condo                77097
Four Family    Four Family           1085
Residential    Condo                26285
               Four Family            617
               Single Family        83957
               Three Family          3108
               Two Family            6567
Single Family  Single Family       279934
Three Family   Three Family          5818
Two Family     Two Family           13493
dtype: int64

In [11]:
#Eliminar valores NaN
df.dropna(subset=['property_type', 'address'  ], axis = 0, inplace = True)

<class 'pandas.core.frame.DataFrame'>
Index: 497960 entries, 0 to 1097627
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   serial_number     497960 non-null  int64         
 1   list_year         497960 non-null  int64         
 2   date_recorded     497960 non-null  datetime64[ns]
 3   town              497960 non-null  object        
 4   address           497960 non-null  object        
 5   assessed_value    497960 non-null  float64       
 6   sale_amount       497960 non-null  float64       
 7   sales_ratio       497960 non-null  float64       
 8   property_type     497960 non-null  object        
 9   residential_type  497960 non-null  object        
 10  assessor_remarks  37626 non-null   object        
 11  opm_remarks       2088 non-null    object        
 12  location          159579 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(7)
memory us

In [21]:
df['assessed_value'].min()

np.float64(0.0)